## Child notebook

This notebook is triggered from the master notebook. this notebook processes the group of files and save them in a temporary parquet folder.

In [ ]:
# print(type(files_list_part))
files_list_part = eval(files_list_part)
print(files_list_part)
print(type(files_list_part))
print(output_path)

In [ ]:
# delete the data file if it exists ( always overwrite scenario)
if mssparkutils.fs.exists(output_path):
  mssparkutils.fs.rm(output_path, recurse=True)

In [ ]:
import time
import timeit
import functools
from pyspark.sql import DataFrame
from pyspark.sql.functions import expr
hash_expression_list = []
start_time = time.time()
output_dfs = []
for file in files_list_part:
    df = spark.read.parquet(file) 
    # if the file_name column is not present then, add the file_name column with the file name that is being processed
    if 'file_name' not in df.columns:
        df = df.withColumn('file_name', expr(f"'{file}'"))
    #get the hashkey
    for i in df.schema:
        if i.name == 'file_name':
            pass
        else:
            name_adjusted = "coalesce(cast("+i.name+" AS STRING),'')"
            hash_expression_list.append(name_adjusted)
    hash_expression_txt = ",'|',".join (hash_expression_list)
    hash_expression_txt = 'md5(concat('+hash_expression_txt+'))'
    df = df.withColumn ('hash_key', expr(hash_expression_txt) )
    output_dfs.append(df)

df_output = functools.reduce(DataFrame.unionAll, output_dfs)
df_output.write.format("parquet").mode("overwrite").save(output_path)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time}")

In [ ]:
df_output.count()

In [ ]:
display(df_output)